In [3]:
import pandas as pd
import matplotlib.pyplot as plt

In [7]:
df = pd.read_csv("synthetic_it_asset_data.csv")

In [8]:
df.head()

,Asset ID,Asset Type,Purchase Date,Lifecycle Status,Maintenance Cost,Replacement Cost
0,A1,Printer,2022-01-26,Retired,4791.18,7617.17
1,A2,Laptop,2019-10-14,In Maintenance,2107.37,7292.71
2,A3,Printer,2024-02-13,Active,179.78,3695.11
3,A4,Router,2019-10-03,Disposed,4988.30,7164.77
4,A5,Router,2021-01-21,Retired,1876.14,9876.08


In [36]:

avg_maintenance_costs = df.groupby('Asset Type')['Maintenance Cost'].mean().reset_index()

df['Maintenance Cost Trend'] = df.groupby('Asset Type')['Maintenance Cost'].transform(lambda x: x.diff().mean())
df['Maintenance Cost Trend'].fillna(0, inplace=True)

avg_maintenance_trends = df.groupby('Asset Type')['Maintenance Cost Trend'].mean().reset_index()

combined_avg = avg_maintenance_costs.merge(avg_maintenance_trends, on='Asset Type')

data = []
for _, row in combined_avg.iterrows():
    asset_type = row['Asset Type']
    avg_maintenance_cost = row['Maintenance Cost']
    avg_maintenance_trend = row['Maintenance Cost Trend']
    data.append((asset_type, avg_maintenance_cost, avg_maintenance_trend))

print(data)


[('Desktop', 2596.28363317757, -0.17819988310928517), ('Laptop', 2524.1061390440723, -0.6314472049689368), ('Printer', 2621.8777863046043, -2.7672179562906036), ('Router', 2561.67573719926, -2.4284444444444473), ('Server', 2549.829240048251, 0.9541641520820764), ('Switch', 2601.7489377934276, 1.5668115091015784)]


In [35]:
df

,Asset ID,Asset Type,Purchase Date,Lifecycle Status,Maintenance Cost,Replacement Cost,Maintenance Cost Trend
0,A1,Printer,2022-01-26,Retired,4791.18,7617.17,-2.767218
1,A2,Laptop,2019-10-14,In Maintenance,2107.37,7292.71,-0.631447
2,A3,Printer,2024-02-13,Active,179.78,3695.11,-2.767218
3,A4,Router,2019-10-03,Disposed,4988.30,7164.77,-2.428444
4,A5,Router,2021-01-21,Retired,1876.14,9876.08,-2.428444
...,...,...,...,...,...,...,...
9995,A9996,Switch,2023-07-23,Active,2065.82,9163.01,1.566812
9996,A9997,Server,2019-12-03,Disposed,2530.88,8886.86,0.954164
9997,A9998,Printer,2024-06-19,Active,1058.48,8011.17,-2.767218
9998,A9999,Printer,2022-06-28,Retired,106.28,8753.65,-2.767218


In [10]:
import psycopg2

In [ ]:
conn = psycopg2.connect(os.getenv('postgres_url'))

In [46]:
create_table_sql = """
CREATE TABLE IF NOT EXISTS maintenance_table (
    id SERIAL PRIMARY KEY,
    asset_type VARCHAR(255) NOT NULL,
    avg_maintenance_cost NUMERIC(10, 2) NOT NULL,
    avg_maintenance_trend NUMERIC(10, 2) NOT NULL
);
"""
cur = conn.cursor()
cur.execute(create_table_sql)

conn.commit()

In [42]:
delete_query = """
DROP TABLE maintenance_cost
"""
cur = conn.cursor()
cur.execute(delete_query)

conn.commit()

In [62]:
insert_query = """
INSERT INTO maintenance_table (asset_type, avg_maintenance_cost, avg_maintenance_trend)
VALUES (%s, %s, %s)
"""
cur = conn.cursor()
try:
    for row in data:
        cur.execute(insert_query, row)
    
    conn.commit()
    print("Data inserted successfully into 'maintenance_table'.")

except Exception as e:
    print(f"Error inserting data: {e}")
    conn.rollback()

finally:
    cur.close()
    conn.close()

Data inserted successfully into 'maintenance_table'.


In [66]:
select_query = """
SELECT * FROM maintenance_table
"""
cur = conn.cursor()
cur.execute(select_query)
rows = cur.fetchall()
conn.commit()

In [67]:
for row in rows:
    print(row[1],row[2],row[3])

Desktop 2596.28 -0.18
Laptop 2524.11 -0.63
Printer 2621.88 -2.77
Router 2561.68 -2.43
Server 2549.83 0.95
Switch 2601.75 1.57
